**Identificação do aluno**

**Email: levi.pereira.junior@ccc.ufcg.edu.br**

**Matrícula: 121210472**

# Análise de Sentimentos em Reviews do IMDb

O principal objetivo desta tarefa é aplicar três modelos de aprendizado de máquina distintos - Regressão Linear, Naive Bayes e Perceptron - para realizar a análise de sentimento em um conjunto de dados de *reviews* de usuários sobre filmes no IMDb. Ao final desta tarefa, você deverá ter uma compreensão mais profunda de como esses modelos funcionam, suas vantagens e limitações quando aplicados a dados textuais do mundo real. Este conjunto de dados inclui avaliações de texto juntamente com rótulos de sentimento correspondentes (positivo ou negativo) para a aprendizagem supervisionada.

## Bibliotecas

In [ ]:
!pip install contractions

In [ ]:
# Para a leitura dos dados
import pandas as pd

# Manipulação de texto
import nltk.corpus
from nltk                             import pos_tag, SnowballStemmer
from nltk.tokenize                    import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import wordnet

# Prepraração dos dados
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.model_selection          import train_test_split

# Modelos de classificação
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.naive_bayes import MultinomialNB

# Avaliação
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

import contractions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Leitura dos dados

In [ ]:
df = pd.read_csv('https://gist.githubusercontent.com/issilva5/44c9406a85b0fed0d62668752cc31b09/raw/49e01d2e8011bdd83d0bc835a518e398ae319303/movie_reviews.csv')
df[['sentiment', 'content']].head()

,sentiment,content
0,neg,now i wont deny that when i purchased this off...
1,neg,the saddest thing about this tribute is that a...
2,neg,last night i decided to watch the prequel or s...
3,neg,i have to admit that i liked the first half of...
4,neg,i was not impressed about this film especially...


Os dados estão distribuídos em duas classes:

In [ ]:
df.groupby('sentiment').count()

,content
sentiment,
neg,12500
pos,12500


A seguir criaremos uma lista com o texto.

In [ ]:
corpus = df['content'].tolist()

## Limpeza dos dados

Antes de aplicar modelos de aprendizado de máquina, você precisará limpar e pré-processar os dados textuais. É esperado que você aplique pelo menos as seguintes tarefas de limpeza dos dados:

- Tokenização
- Remoção de palavras muito pequenas (<= 2) ou muito grandes (>= 15).
- Remoção de stopwords.
- Stemming.

In [ ]:
stop_words = nltk.corpus.stopwords.words("english")
stemmer = SnowballStemmer("english")
lemmatizer = nltk.stem.WordNetLemmatizer()

def remove_palavras_pequenas_grandes(tokens):
  stopwords_removidas = [token.lower() for token in tokens if len(token) > 2 and len(token) <= 15]
  return stopwords_removidas

def remove_stopwords(tokens):
  nao_stop_words = [token for token in tokens if token.lower() not in stop_words]
  return nao_stop_words

def stemming(tokens):
  palavras_stemmizadas = [stemmer.stem(token) for token in tokens]
  return palavras_stemmizadas

def corrigir_documento(document):
  return contractions.fix(document)

def get_wordnet_pos(treebank_tag):
  if treebank_tag.startswith('J'): # Adjetivo
    return wordnet.ADJ
  elif treebank_tag.startswith('V'): # Verbo
    return wordnet.VERB
  elif treebank_tag.startswith('N'): # Substantivo
    return wordnet.NOUN
  elif treebank_tag.startswith('R'):# Advérbio
    return wordnet.ADV
  else:
    return None

def lemmatize(tokens):
  lemmatized_tokens = []
  pos_tagged = pos_tag(tokens)
  for token, tag in pos_tagged:
    wordnet_pos = get_wordnet_pos(tag) or wordnet.NOUN
    lemmatized_tokens.append(lemmatizer.lemmatize(token, pos=wordnet_pos))

  return lemmatized_tokens

In [ ]:
def process_corpus(corpus):
  corpus_processed = []
  for document in corpus:
    document = corrigir_documento(document)
    tokens = word_tokenize(document)
    tokens = remove_stopwords(tokens)
    tokens = remove_palavras_pequenas_grandes(tokens)
    tokens = lemmatize(tokens)
    tokens = stemming(tokens)
    corpus_processed.append(" ".join(tokens))
  return corpus_processed

corpus_processed = process_corpus(corpus)

## Preparando os dados para os modelos

Primeiramente, realize a vetorização TF-IDF dos dados.

In [ ]:
# Você deve instanciar um vetorizador TF-IDF e aplicá-lo ao corpus processado.

x_treino, x_teste, y_treino, y_teste = train_test_split(corpus_processed, df['sentiment'], test_size=0.2, shuffle=True)

vetorizador = TfidfVectorizer(max_df=0.95, min_df=1, ngram_range=(1,2), max_features=None)

Agora, realize a partição treino e teste dos dados.

In [ ]:
# Você deve realizar a partição treino e teste dos dados aqui.

x_treino_vetorizado = vetorizador.fit_transform(x_treino)
x_teste_vetorizado = vetorizador.transform(x_teste)

print(corpus_processed[0])
x_teste

## Treinando modelos

Finalmente, instancie e treine os três modelos (Regressão Logística, Naive Bayes e Perceptron).

In [ ]:
# Você deve instanciar e treinar os três modelos de Regressão Logística

modelo_NaiveBayes = MultinomialNB(alpha=0.1)
modelo_NaiveBayes.fit(x_treino_vetorizado, y_treino)
predicao_NaiveBayes = modelo_NaiveBayes.predict(x_teste_vetorizado)

modelo_RegressaoLogistica = LogisticRegression()
modelo_RegressaoLogistica.fit(x_treino_vetorizado, y_treino)
predicao_RegressaoLogistica = modelo_RegressaoLogistica.predict(x_teste_vetorizado)

modelo_Perceptron = Perceptron()
modelo_Perceptron.fit(x_treino_vetorizado, y_treino)
predicao_Perceptron = modelo_Perceptron.predict(x_teste_vetorizado)

## Avaliação dos modelos

Você deve realizar as predições para cada um dos três modelos.

In [ ]:
# Realize as predições para cada um dos modelos

def classificar_sentimento(vetorizador, classificador, entrada):
  resultado = ""

  if (entrada != ""):
    corpus = [entrada]
    corpus_processados = process_corpus(corpus)
    predicao_y = vetorizador.transform(corpus_processados)
    predicao_classificador = classificador.predict(predicao_y)
    if predicao_classificador[0] == "pos":
      resultado = "Positivo"
    elif (predicao_classificador[0] == "neg"):
      resultado = "Negativo"

  return resultado

def classificar_exemplo(exemplo, esperado):
  resultado_NaiveBayes = classificar_sentimento(vetorizador, modelo_NaiveBayes, exemplo)
  resultadoRegressaoLogistica = classificar_sentimento(vetorizador, modelo_RegressaoLogistica, exemplo)
  resultado_Perceptron = classificar_sentimento(vetorizador, modelo_Perceptron, exemplo)

  print(f'Classificando o exemplo: "{exemplo1}"\nResultado esperado: {esperado}')

  print(f"\nResultado através da classificação Naive Bayes: {resultado_NaiveBayes}")
  print(f"Resultado através da classificação Regressão Logística: {resultadoRegressaoLogistica}")
  print(f"Resultado através da classificação Perceptron: {resultado_Perceptron}\n")

exemplo1 = "I absolutely loved the new movie I watched last night. The acting was superb, and the storyline was captivating from start to finish."
exemolo2 = "The service at the restaurant was terrible. The staff was rude, and the food was cold and tasteless. I won't be coming back."

classificar_exemplo(exemplo1, "Positivo")
classificar_exemplo(exemolo2, "Negativo")

Plote a matriz de confusão para cada modelo.

In [ ]:
# Faça o plot da matriz de confusão de contagem para cada um dos modelos.
import seaborn as sns

def plot_matriz_confusao(y_true, y_pred, modelo, titulo):
  matriz_confusao = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=modelo.classes_)
  sns.heatmap(matriz_confusao, annot=True, fmt='d', cmap='Blues', xticklabels=modelo.classes_, yticklabels=modelo.classes_)
  plt.title(f'Matriz de Confusão - {titulo}')
  plt.show()

plot_matriz_confusao(y_teste, predicao_NaiveBayes, modelo_NaiveBayes, "Naive Bayes")
plot_matriz_confusao(y_teste, predicao_RegressaoLogistica, modelo_RegressaoLogistica, "Regressão Logistica")
plot_matriz_confusao(y_teste, predicao_Perceptron, modelo_Perceptron, "Perceptron")

Calcule métricas (acurácia, recall, precision, f1-score) para cada uma das predições.

In [ ]:
# Para cada um dos modelos produza as métricas pedidas.

print("\nPara o modelo Naive Bayes:\n")
print(classification_report(y_teste, predicao_NaiveBayes))
print("\nPara o modelo Regressão Logistica:\n")
print(classification_report(y_teste, predicao_RegressaoLogistica))
print("\nPara o modelo Perceptron:\n")
print(classification_report(y_teste, predicao_Perceptron))

#### Perguntas

***Discuta sobre os resultados do modelo considerando as matrizes de confusão e as métricas calculadas.***

Observa-se que o modelo Regressão Logistica foi o melhor entre os três modelos.
Tanto é que a F1 (relação harmônica entre o precision e recall) foi melhor para a Regressão Logistica.

Para o modelo Naive Bayes:
- Precision: 0.87
- Recall: 0.87
- F1: 0.87

Para o modelo Regressão Logistica:
- Precision: 0.90
- Recall: 0.87
- F1: 0.89

Para o modelo Perceptron:
- Precision: 0.86
- Recall: 0.88
- F1: 0.87


## Interpretando os modelos

Uma subárea importante da aprendizagem de máquina é a interpretação dos modelos.

Nesta parte do laboratório, você deve implementar funções para facilitar a interpretação dos modelos treinados e responder algumas perguntas sobre eles.

### Implementando funções auxiliares

A seguir é pedido que você implemente duas funções.

A primeira delas **recupera_palavras_positivas** deve retornar as top-20 palavras que mais contribuem para a classificação do texto como positivo. A segunda **recupera_palavras_negativas** deve fazer o equivalente para a classificação negativa.

Os modelos de Regressão Logística e Perceptron tem um parâmetro chamado ***coef_*** este parâmetro retorna o peso de cada feature (palavra) tem no modelo. Palavras com peso positivo influenciam para a classificação positiva, e palavras com peso negativo fazem o inverso. O valor desse parâmetro tem a dimensão (1, n_features).

Já o modelo Naive Bayes tem um parâmetro chamado ***feature_log_prob_***. Este parâmetro retorna o log das probabilidades de cada palavra aparecer no texto dada uma classe. O valor desse parâmetro tem a dimensão (2, n_features), de modo que a posição 0 corresponde as probabilidades para a classe negativa e a posição para a classe positiva. Quanto maior for a probabilidade de uma palavra, maior podemos dizer que é sua influência na classificação.

Para acessar o nomes das features em ordem utilize o método **get_feature_names_out** do vetorizador construído.

Ambas as funções devem retornar uma lista de tuplas (string, float), ou seja, (palavra, peso).

In [ ]:
import numpy as np

def recupera_palavras_positivas(modelo):
  try:
    coeficientes = modelo.coef_[0]
  except AttributeError:
    coeficientes = modelo.feature_log_prob_[1]

  top_positivas = np.argsort(coeficientes)[-20:]
  palavras_positivas = [vetorizador.get_feature_names_out()[i] for i in top_positivas]

  return palavras_positivas

def recupera_palavras_negativas(modelo):
  try:
    coeficientes = modelo.coef_[0]
  except AttributeError:
    coeficientes = modelo.feature_log_prob_[1]

  top_negativas = np.argsort(coeficientes)[:20]
  palavras_negativas = [vetorizador.get_feature_names_out()[i] for i in top_negativas]

  return palavras_negativas

### Visualizando e interpretando

Use a função abaixo para visualizar uma nuvem de palavras do retorno das funções.

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def plot_wordcloud(df, color="white"):
  words = ' '.join(df)
  cleaned_word = ' '.join([word for word in words.split()])
  wordcloud = WordCloud(background_color=color, width=2500, height = 2500).generate(cleaned_word)

  plt.figure(1, figsize = (10,7))
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.show()

#### Palavras positivas

In [ ]:
# Plote aqui a nuvem de palavras para a Regressão Logística

plot_wordcloud(recupera_palavras_positivas(modelo_RegressaoLogistica))

In [ ]:
# Plote aqui a nuvem de palavras para o Naive Bayes

plot_wordcloud(recupera_palavras_positivas(modelo_NaiveBayes))

In [ ]:
# Plote aqui a nuvem de palavras para o Perceptron

plot_wordcloud(recupera_palavras_positivas(modelo_Perceptron))

#### Palavras negativas

In [ ]:
# Plote aqui a nuvem de palavras para a Regressão Logística

plot_wordcloud(recupera_palavras_negativas(modelo_RegressaoLogistica))

In [ ]:
# Plote aqui a nuvem de palavras para o Naive Bayes

plot_wordcloud(recupera_palavras_negativas(modelo_NaiveBayes))

In [ ]:
# Plote aqui a nuvem de palavras para o Perceptron

plot_wordcloud(recupera_palavras_negativas(modelo_Perceptron))

#### Perguntas

**1. Analisando as nuvens de palavras positivas de cada modelo, é possível identificar que as palavras estão associadas à um sentimento positivo? Dê exemplos.**

Sim, é possível identificar.
O modelo de Regressão Logistica, Naive Bayes e Perceptron encontraram várias entre palavras as top 20 que estão mais associadas a sentimentos positivos.

Exemplos de palavras com palavras associadas a un sentimento positivo para
- Regressão Logistica: best, superb, amaze e brilliant.
- Naive Bayes: love, good e greet.
- Perceptron: amazing, incredible e excellent.

Entretanto, o modelo de Regressão Logistica e Naive Bayes encontraram não só palavras com sentimentos positivo, mas tambmém algumas palavras que por si só não podem ser classificadas como positivas.

Como por exemplo:
- Regressão Logistica: one, today e still.
- Naive Bayes: character, movie e film.


**2. Analisando as nuvens de palavras negativas de cada modelo, é possível identificar que as palavras estão associadas à um sentimento negativo? Dê exemplos.**

Sim, é possivel identificar palavras associadas a um sentimento negativo em todos os modelos. Temos para:
- Regressão Logistica: terrible, poor, bare e horrible.
- Naive Bayes: bad e horror.
- Perceptron: awful, poor e bad.

Entretanto, o modelo de Naive Bayes encontrarou não só palavras com sentimentos negativos, mas tambmém algumas palavras que por si só não podem ser classificadas como negativas.

Como por exemplo:
Naive Bayes: box, plot, even e one.

**3. Considerando as métricas calculadas e a análise acima, qual modelo você acredita ser o melhor na tarefa? Por quê?**

Acredito que a Regressão Logistica seja a melhor porque essas palavras se assemelham mais as palavras positivas e negativas do que o Perceptron e Naive Bayes.
Nota-se que ele da maior peso as palavras que realmente são de sentimento positivo e neghativo do que os outros dois modelos de aprendizagem supervisionada.

Inclusive, a regressãõ logistica teve maiores métricas entre todos, inclusive na harmonia entre precisão e recall, a chamada F1.

## Testando mais modelos

Até então no exerício utilizamos três modelos de classificação, entretanto existem muitos outros disponíveis no sklearn. Escolha um dos três modelos para treinar e avaliar nos dados. Em seguida compare os resultados com os modelos anteriores.

- [Nearest Neighbors Classification](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification)
- [Decision Tree](https://scikit-learn.org/stable/modules/tree.html#classification)
- [Support Vector Machine](https://scikit-learn.org/stable/modules/svm.html#classification)

In [ ]:
# Importe o modelo que será utilizado

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [ ]:
# Treine aqui o modelo escolhido

modelo_KNN = KNeighborsClassifier(n_neighbors=5)
modelo_KNN.fit(x_treino_vetorizado, y_treino)
predicao_KNN = modelo_KNN.predict(x_teste_vetorizado)

modelo_DecisionTree = DecisionTreeClassifier()
modelo_DecisionTree.fit(x_treino_vetorizado, y_treino)
predicao_DecisionTree = modelo_DecisionTree.predict(x_teste_vetorizado)

modelo_SVM = SVC()
modelo_SVM.fit(x_treino_vetorizado, y_treino)
predicao_SVM = modelo_SVM.predict(x_teste_vetorizado)

In [ ]:
# Faça a previsão dos resultados
print("Para o modelo KNN:\n")
print(classification_report(y_teste, predicao_KNN))
print("\n\nPara o modelo Decision Tree:\n")
print(classification_report(y_teste, predicao_DecisionTree))
print("\n\nPara o modelo SVM:\n")
print(classification_report(y_teste, predicao_SVM))

In [ ]:
# Avalie o modelo usando a matriz de confusão e as métricas anteriores

plot_matriz_confusao(y_teste, predicao_KNN, modelo_KNN, "KNN")
plot_matriz_confusao(y_teste, predicao_DecisionTree, modelo_DecisionTree, "Decision Tree")
plot_matriz_confusao(y_teste, predicao_SVM, modelo_SVM, "SVM")

### Perguntas

**1. Explique brevemente como o modelo que você escolheu funciona (a documentação do sklearn pode servir de fonte para esta resposta).**

*K-Nearest Neighbors (KNN)*:
O K-Nearest Neighbors (KNN) é um classificador baseado em instâncias que faz previsões com base na similaridade entre amostras. O modelo armazena todas as instâncias de treinamento e, para fazer uma previsão, encontra os k vizinhos mais próximos à nova amostra e decide a classe com base na maioria das classes desses vizinhos. O parâmetro n_neighbors define quantos vizinhos considerar. A escolha do valor de k pode impactar a performance do modelo, e valores pequenos podem levar a overfitting, enquanto valores grandes podem suavizar demais as decisões.

*Decision Tree*:
A Decision Tree é um classificador baseado em uma estrutura de árvore onde cada nó interno representa uma decisão baseada em um atributo, e cada folha representa uma classe. O modelo divide os dados em subsets menores e mais homogêneos através de regras de decisão, otimizando um critério de divisão (como a impureza de Gini ou a entropia) para determinar a melhor divisão. As árvores de decisão são intuitivas e fáceis de interpretar, mas podem ser propensas ao overfitting se não forem podadas adequadamente.

*Support Vector Machine (SVM)*:
A Support Vector Machine (SVM) é um classificador que busca encontrar um hiperplano que melhor separa os dados em diferentes classes. O objetivo é maximizar a margem entre as classes, que é a distância entre o hiperplano e os pontos de dados mais próximos (vetores de suporte). SVM é eficaz em espaços de alta dimensão e em casos onde o número de dimensões é maior que o número de amostras. O parâmetro C controla o trade-off entre maximizar a margem e minimizar o erro de classificação. O parâmetro kernel define a função usada para transformar os dados em um espaço de características superior onde a separação linear é possível.


**2. Como foi o desempenho do modelo escolhido em relação aos demais modelos?**

O SVM saiu melhor do que os demais quando falamos em precision, recall e a F1.